In [19]:
import os
import uuid
import pandas as pd
import mlflow

In [20]:
year = 2023
month = 1
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/prediction_{year:04d}-{month:02d}.parquet'

print(f'Input file: {input_file}')
print(f'Output file: {output_file}')

Input file: https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Output file: output/green/prediction_2023-01.parquet


In [21]:
# temporary solution to see environment variables when running Jupyter Notebook from IDE
os.environ["MLFLOW_EXPERIMENT_NAME"] = "module_4_nyc_taxi_duration_prediction_with_sklearn_pipeline_experiment"
os.environ["RUN_ID"] = "8d6b2c8289b94d1cb40a313e0bf92aca"

os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio_user"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio_password"

In [22]:
MLFLOW_TRACKING_URI = os.environ.get("MLFLOW_TRACKING_URI")
print("Environment variables for MLFlow: ", MLFLOW_TRACKING_URI)

AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
MLFLOW_S3_ENDPOINT_URL = os.environ.get("MLFLOW_S3_ENDPOINT_URL")
print("Environment variables for S3: ", MLFLOW_S3_ENDPOINT_URL, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

MLFLOW_EXPERIMENT_NAME = os.environ.get("MLFLOW_EXPERIMENT_NAME")
RUN_ID = os.environ.get("RUN_ID")
print("Environment variables for model: ", MLFLOW_EXPERIMENT_NAME, RUN_ID)

Environment variables for MLFlow:  http://localhost:5000
Environment variables for S3:  http://localhost:9000 minio_user minio_password
Environment variables for model:  module_4_nyc_taxi_duration_prediction_with_sklearn_pipeline_experiment 8d6b2c8289b94d1cb40a313e0bf92aca


In [23]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1750103452800, experiment_id='1', last_update_time=1750103452800, lifecycle_stage='active', name='module_4_nyc_taxi_duration_prediction_with_sklearn_pipeline_experiment', tags={}>

In [24]:
def generate_ride_ids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

In [26]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    df['ride_id'] = generate_ride_ids(len(df))

    return df

In [27]:
def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [28]:
def load_model(run_id):
    model_uri = f's3://bucket/1/{run_id}/artifacts/model'
    print(f'Loading model using uri: {model_uri}')
    model = mlflow.pyfunc.load_model(model_uri)
    return model

In [29]:
def prepare_results(df, y_pred):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    df_result['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime']
    df_result['PULocationID'] = df['PULocationID']
    df_result['DOLocationID'] = df['DOLocationID']
    df_result['trip_distance'] = df['trip_distance']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    return df

In [30]:
def apply_model(input_file, run_id, output_file):
    df = read_dataframe(input_file)
    dicts = prepare_dictionaries(df)
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = prepare_results(df, y_pred)
    df_result.to_parquet(output_file, engine='pyarrow', index=False)

In [32]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)

Loading model using uri: s3://bucket/1/8d6b2c8289b94d1cb40a313e0bf92aca/artifacts/model


In [ ]:
! ls output/green/

In [33]:
output_df = pd.read_parquet(output_file)
output_df.head()


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration,ride_id,PU_DO
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,...,0.0,None,1.0,24.18,1.0,1.0,2.75,11.016667,4a2c082b-131f-4f07-832f-1562a107d283,166_143
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,...,0.0,None,1.0,15.84,1.0,1.0,0.00,6.766667,98725173-afdd-4fc8-9511-48c185f90a1c,24_43
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,...,0.0,None,1.0,11.64,1.0,1.0,0.00,6.333333,ec6d0205-f954-4c2a-86c1-5e02fb6e727c,223_179
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,...,0.0,None,1.0,10.20,1.0,1.0,0.00,5.816667,7e4cf721-dbdb-4d16-a69a-ea3e03d5b3a8,41_238
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,...,0.0,None,1.0,8.00,1.0,1.0,0.00,5.966667,682b3e2f-5aca-4e8b-b063-7776748fe47d,41_74
